In [1]:
import pandas as pd
from T_data_loader import *

C:\Users\pierr\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Data preprocessing

This notebook aims to explain the preprocessing steps in order to get the datasets that will be used for the analysis, repectively "Actor.pkl" and "Movie.pkl" <br> All these steps are indicative, you don't have to run all these cells as running all the notebook a quite time-consuming (around 7 hours due to the part "Data from Web").

## IMDb datasets

First, you can go to https://datasets.imdbws.com, download the files and extract them :
- title.basics.tsv.gz
- title.crew.tsv.gz
- title.ratings.tsv.gz
- name.basics.tsv.gz
  
<br> Note that a descriptive of this dataset can be found on https://developer.imdb.com/non-commercial-datasets/ or in the README in https://github.com/epfl-ada/ada-2024-project-importnumpyaspd/tree/main/data/IMDb.

In [2]:
### Add the path of the folder you have just downloaded
# the folder containing all datasets on your machine :
path = ""

Custom column names are defined :

In [3]:
NAMES_CREW = ['IMDb_title_ID','IMDb_director_ID', 'IMDb_writers_ID']
NAMES_ratings = ['IMDb_title_ID', 'Average rating', 'number of votes'] 
NAMES_BASICS = ['IMDb_people_ID', 'Name', 'birthYear', 'deathYear', 'profession', 'knownForTitles']
NAMES_TITLES = ['IMDb_title_ID', 'TitleType', 'Primary_title', 'Original_title', 'isAdult', 'release_date', 'end_year', 'runtime', 'genres']

All datasets are loaded :

In [4]:
df_IMDb_crew = pd.read_csv(path+"title.crew.tsv/title.crew.tsv", sep='\t', names = NAMES_CREW, header = 0)
df_IMDb_ratings = pd.read_csv(path+"title.ratings.tsv/title.ratings.tsv", sep='\t', names = NAMES_ratings, header = 0)
df_IMDb_title = pd.read_csv(path+"title.basics.tsv/title.basics.tsv", sep='\t', names = NAMES_TITLES, header = 0, low_memory=False)
df_IMDb_name = pd.read_csv(path+"name.basics.tsv/name.basics.tsv", sep='\t', names = NAMES_BASICS, header = 0)

For title dataset, only movie must be selected (drop series,...). Without this set, df_IMDb_title is huge.
<br> Few unecessary columns are also dropped.

In [5]:
selected_type = 'movie'
df_IMDb_title = df_IMDb_title.query('TitleType==@selected_type')
df_IMDb_title=df_IMDb_title[['IMDb_title_ID', 'release_date','runtime','Primary_title', 'Original_title']]

### Merging Titles, crews and ratings :

In [6]:
print(df_IMDb_title.shape)
print(df_IMDb_ratings.shape)
print(df_IMDb_crew.shape)

(696953, 5)
(1497560, 3)
(10561467, 3)


The ratings and crew dataset contains also ratings of non-movie type. We merge on title datset as we have already selected the movie's row. (here, how = left)

In [7]:
IMDb_title_rating = pd.merge(df_IMDb_title, df_IMDb_ratings, how='left', on = 'IMDb_title_ID' )

In [8]:
IMDb = pd.merge(IMDb_title_rating, df_IMDb_crew, how='left', on = 'IMDb_title_ID' )

## CMU "Per title" dataset

You can go to https://www.cs.cmu.edu/~ark/personas/, download the dataset and extract all files. This webpage provide also a descriptive of the datasets. A more detailed description figures in the README of the download folder or in https://github.com/epfl-ada/ada-2024-project-importnumpyaspd/tree/main/data/CMU.

We are interested in the movie.metadata file :

In [9]:
# custom column names :
NAMES_MOVIES = ['Wikipedia_movie_ID','Freebase_movie_ID','Movie_name','Movie_release_date','Movie_box_office_revenue','Movie_runtime','Movie_languages','Movie_countries','Movie_genres']

In [10]:
# load the dataset
df_CMU_movies = pd.read_csv(path+"MovieSummaries/movie.metadata.tsv", sep='\t', names = NAMES_MOVIES, header = 0)

## Merging CMU "Per title" and IMDb

### Using unique index

We will try to merge both datsets using a unique index. Few combination of columns have been tried to achieve a satisfying result. Here we show our final choice that where the new index is composed of the name and the release year of the movies. So we supposed that these 2 informations were necessary to distinguish between every movies. As we will see in the following cells it was somethimes not sufficient.

We keep only the year as release date : 

In [11]:
# For CMU
df_CMU_movies["Movie_release_date"]=pd.to_datetime(df_CMU_movies["Movie_release_date"], format='mixed', errors='coerce').dt.year.astype('Int64')
# For IMDb
IMDb['release_date']=pd.to_datetime(IMDb['release_date'], format='mixed', errors='coerce').dt.year.astype('Int64')

A new column "modified name" is created where we drop the ponctuation and the space. All characters are also in lowercase.

In [12]:
# For CMU
df_CMU_movies_modified_title = df_CMU_movies.Movie_name.str.replace(r'[^\w\s]', '', regex=True).str.replace(r'\s+', '', regex=True).str.lower()
df_CMU_movies["modified_title"]=df_CMU_movies_modified_title
# For IMDb
df_IMDb_modified_title = IMDb.Primary_title.str.replace(r'[^\w\s]', '', regex=True).str.replace(r'\s+', '', regex=True).str.lower()
IMDb["modified_title"]=df_IMDb_modified_title

(copy for later)

In [13]:
init_IMDb_copy = IMDb.copy()
init_CMU_copy = df_CMU_movies.copy()

All rows with NAN in Title name / date are dropped :

In [14]:
# For CMU
s1 = df_CMU_movies.shape[0]
df_CMU_movies = df_CMU_movies.dropna(subset=['Movie_release_date'])
s2 = df_CMU_movies.shape[0]
print(f'{s1-s2} rows are lost with this operation in CMU dataset.  / {s1} ')
# For IMDb
s1 = IMDb.shape[0]
IMDb = IMDb.dropna(subset=['release_date'])
s2 = IMDb.shape[0]
print(f'{s1-s2} rows are lost with this operation in IMDb dataset. / {s1}')

6903 rows are lost with this operation in CMU dataset.  / 81740 
101376 rows are lost with this operation in IMDb dataset. / 696953


In [15]:
# Store a dataset with row with NAN as Title or release date
# This step is done in prevision of the section "Merging using Data from the web".
CMU_with_NAN = init_CMU_copy[~init_CMU_copy.index.isin(df_CMU_movies.index)]
print(f'Quick check of the size of the CMU containing the NAN values : {CMU_with_NAN.shape[0]}')

Quick check of the size of the CMU containing the NAN values : 6903


Defining new index names based on release date and title : 

In [16]:
# For CMU
new_id_CMU = df_CMU_movies.Movie_release_date.astype(str)+df_CMU_movies.modified_title
# For IMDb
new_id_IMDb = IMDb.release_date.astype(str)+IMDb.modified_title

Once we get the new index, we check that this is a unique index :

In [17]:
# For CMU
CMU_movies_newind = df_CMU_movies.copy()
CMU_movies_newind.index = new_id_CMU
print(f'CMU has a unique indexing : {CMU_movies_newind.index.is_unique}')
# For IMDb
IMDb_movies_newind = IMDb.copy()
IMDb_movies_newind.index = new_id_IMDb
print(f'IMDb has a unique indexing : {IMDb_movies_newind.index.is_unique}')

CMU has a unique indexing : False
IMDb has a unique indexing : False


Unfortunately we have a non unique indexing.<br>Let's count how much rows have the same index :

In [18]:
mask_duplicate = CMU_movies_newind.index.duplicated(keep=False)
df_CMU_movies_wo_dupl = CMU_movies_newind[~mask_duplicate]

s1 = CMU_movies_newind.shape[0]
s2 = df_CMU_movies_wo_dupl.shape[0]

print(f'{s1-s2} rows are lost with this operation in CMU dataset  / {s1} ')
print(f'CMU has a unique indexing : {df_CMU_movies_wo_dupl.index.is_unique}')

275 rows are lost with this operation in CMU dataset  / 74837 
CMU has a unique indexing : True


In [19]:
mask_duplicate = IMDb_movies_newind.index.duplicated(keep=False)
df_IMDb_movies_wo_dupl = IMDb_movies_newind[~mask_duplicate]

s1 = IMDb_movies_newind.shape[0]
s2 = df_IMDb_movies_wo_dupl.shape[0]

print(f'{s1-s2} rows are lost with this operation in IMDb dataset / {s1}')
print(f'IMDb has a unique indexing : {df_IMDb_movies_wo_dupl.index.is_unique}')

11132 rows are lost with this operation in IMDb dataset / 595577
IMDb has a unique indexing : True


As the amount of duplicates are small, we decide to drop all of them for this step.

#### Merge CMU with IMDb :

In [20]:
merged = df_IMDb_movies_wo_dupl.merge(df_CMU_movies_wo_dupl, left_index=True, right_index=True, how='inner')

In [21]:
print(f'A total of {merged.shape[0]} out of {init_CMU_copy.shape[0]} have a match')

A total of 44864 out of 81740 have a match


Previously we used the "primary title" of the IMDb dataset to create the "modified_title" columns. Let's redo the previous operation on the "original title" column of the IMDb dataset.

We don't start with the entire dataset, but we take the unmatched rows of each dataset :

In [22]:
notmatched_CMU = df_CMU_movies_wo_dupl[~df_CMU_movies_wo_dupl.index.isin(df_IMDb_movies_wo_dupl.index)]
notmatched_IMDb = IMDb_movies_newind[~IMDb_movies_newind.index.isin(df_CMU_movies_wo_dupl.index)]

print(notmatched_IMDb.shape)
print(notmatched_CMU.shape)

(549793, 10)
(29698, 10)


(Same procedure as before but with Original_title)

In [23]:
notmatched_IMDb = notmatched_IMDb.drop(columns=["modified_title"])
df_IMDb_modified_titlev2 = notmatched_IMDb.Original_title.str.replace(r'[^\w\s]', '', regex=True).str.replace(r'\s+', '', regex=True).str.lower()
notmatched_IMDb["modified_title"]=df_IMDb_modified_titlev2
new_id_IMDbv2 = notmatched_IMDb.release_date.astype(str)+notmatched_IMDb.modified_title

notmatched_IMDb_newind = notmatched_IMDb.copy()
notmatched_IMDb_newind.index = new_id_IMDbv2

mask_duplicate = notmatched_IMDb_newind.index.duplicated(keep=False)
notmatched_IMDb_wo_dupl = notmatched_IMDb_newind[~mask_duplicate]

print(f'{s1-s2} rows are lost with this operation in IMDb dataset / {s1}')
print(f'IMDb has a unique indexing : {notmatched_IMDb_wo_dupl.index.is_unique}')

11132 rows are lost with this operation in IMDb dataset / 595577
IMDb has a unique indexing : True


We merge the 2 datasets of unmatched rows (that also have unique index) :

In [24]:
mergedv2 = notmatched_IMDb_wo_dupl.merge(notmatched_CMU, left_index=True, right_index=True, how='inner')
print(f'We add {mergedv2.shape[0]} rows using the original title !')

We add 3115 rows using the original title !


We then concat both merge :

In [25]:
merge_final = pd.concat([merged,mergedv2])
print(f'Merge dataset has unique indexing : {merge_final.index.is_unique}')
print(f'Size of dataset : {merge_final.shape}')

Merge dataset has unique indexing : True
Size of dataset : (47979, 20)


As se can see, a non negligeble part of CMU dataset has not find a match : 

In [26]:
#CMU
notmatched_CMU2 = notmatched_CMU[~notmatched_CMU.index.isin(notmatched_IMDb_wo_dupl.index)]
## add also the part of CMU that was dropped since they have a NAN in column "title or release date"
notmatched_CMU2 = pd.concat([CMU_with_NAN,notmatched_CMU2])
#IMDb
notmatched_IMDb2 = notmatched_IMDb_wo_dupl[~notmatched_IMDb_wo_dupl.index.isin(notmatched_CMU.index)]

print(f'Number of unmatched rows of CMU dataset : {notmatched_CMU2.shape}')

26583
6903
33486
Number of unmatched rows of CMU dataset : (33486, 10)


In [29]:
notmatched_CMU2.head()

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres,modified_title
14,11250635,/m/02r52hc,The Mechanical Monsters,<NA>,NaN,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06n90"": ""Science Fiction"", ""/m/03k9fj"": ""...",themechanicalmonsters
73,31137877,/m/0gh7n22,Boadicea,<NA>,NaN,NaN,{},{},"{""/m/07s9rl0"": ""Drama"", ""/m/03hn0"": ""Historica...",boadicea
80,27374355,/m/0by1_ff,Les Indiens sont encore loin,<NA>,NaN,95.0,{},"{""/m/0f8l9c"": ""France"", ""/m/06mzp"": ""Switzerla...","{""/m/07s9rl0"": ""Drama""}",lesindienssontencoreloin
102,29766415,/m/0fp_syp,Donald's Crime,<NA>,NaN,NaN,{},{},"{""/m/02hmvc"": ""Short Film""}",donaldscrime
119,28415406,/m/0crj1f3,The Last Trackers of the Outback,<NA>,NaN,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/0f8l9c"": ""France"", ""/m/0chghy"": ""Australia""}","{""/m/0jtdp"": ""Documentary""}",thelasttrackersoftheoutback


Next section will try a other way to merge these 2 datasets.

As the amount of duplicates respectively to their index (release year + name) are almost negligible, we don't take care of adding them in the "notmatched_CMU2" and "notmatched_IMDb2" for the next step.

### Merging using Data from the web

We try now to get the IMDb id of the movie contained in CMU dataset by scraping wikipedia. This method seems to be really precise to merge both dataset, but it was not consider in the first step due to it's computational time. Now that we've reduced the number of samples, let's see what we can do.

In [30]:
import requests
from bs4 import BeautifulSoup
import re
import urllib.parse as url
from tqdm import tqdm

The function "get_IMDb_id" do :
- go to wikipedia page of the film using the wikipedia id
- search in the HTML text for an URL that starts with "https://www.imdb.com/title/tt". (This is the imbd URL of the film)
- if there is a single adresse looking like this in the HTML page it return last part of the URL ("A part of path") that correspond to the IMDb id.

In [31]:
def get_IMDb_id(wikipedia_id):
    # sources : 
    # - https://www.geeksforgeeks.org/beautifulsoup-scraping-link-from-html/
    # - https://stackoverflow.com/questions/7253803/how-to-get-everything-after-last-slash-in-a-url
    r = requests.get("https://en.wikipedia.org/?curid="+str(wikipedia_id))
    
    if r.status_code == 404:
        return pd.NA
        
    soup = BeautifulSoup(r.content, "html.parser")
    nbr_link = 0
    for link in soup.find_all('a',attrs={'href': re.compile("^https://www.imdb.com/title/tt")}):
        nbr_link += 1
        href  = link.get('href')
        url_parts = url.urlparse(href)
        IMBd_id = url_parts.path.split('/')[2]
    if nbr_link==1:
        return IMBd_id
    else :
        return pd.NA

In [32]:
notmatched_CMU2_copy = notmatched_CMU2.copy()
notmatched_CMU2_copy=notmatched_CMU2_copy['Wikipedia_movie_ID'].astype(str)

In [33]:
tqdm.pandas()
notmatched_CMU2_copy = notmatched_CMU2_copy.progress_apply(lambda x: get_IMDb_id(x))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26583/26583 [6:40:28<00:00,  1.11it/s]


Finally we can merge the notmatched dataset and concat it with the merge_dataset that we got using unique indexing method.

In [41]:
notmatched_CMU2['IMDb_title_ID'] = notmatched_CMU2_copy
mergev3 = pd.merge(notmatched_IMDb2, notmatched_CMU2, how = 'inner', on = 'IMDb_title_ID' )
print(f'{mergev3.shape[0]} new samples have been matched with web scraping !')
final = pd.concat([merge_final,mergev3])
print("Final score :")
print(f'{final.shape[0]} out of {init_CMU_copy.shape[0]} have been matched')

/var/folders/37/w8_bl_xd53n7dlt9q3m5v8vm0000gn/T/ipykernel_27614/483739325.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notmatched_CMU2['IMDb_title_ID'] = notmatched_CMU2_copy


9753 new samples have been matched with web scraping !
Final score :
57732 out of 81740 have been matched


## CMU actor dataset

Reminder <br>
You can go to https://www.cs.cmu.edu/~ark/personas/, download the dataset and extract all files. This webpage provide also a descriptive of the datasets. A more detailed description figures in the README of the download folder or in https://github.com/epfl-ada/ada-2024-project-importnumpyaspd/tree/main/data/CMU.

We are interested now in the character.metadata file :

In [34]:
# custom column names :
character_column = ['Wikipedia_movie_ID','Freebase_movie_ID','Movie_release_date','Character_name','actor_DOB','actor_gender','actor_height','actor_ethnicity','actor_name','actor_age_atmovierelease','freebase_character_actor_map_id','freebase_character_id','freebase_actor_id']

In [36]:
# load the dataset
CMU_character = pd.read_csv(path+"/MovieSummaries/character.metadata.tsv", sep='\t',names = character_column, header = 0)

We are interested to get a serie that have in index the name of an actor and contain a list of all the film that he plays in.

We have done it using groupby and apply methods : 

In [37]:
def get_films(df):
    list_film = df.Freebase_movie_ID.tolist()
    return list_film

In [38]:
actor = CMU_character.groupby(by='freebase_actor_id').apply(get_films,include_groups=False)

## Save final dataset 

All dataset are ready to be used for the analysis. Let's save them in pickle files.

In [39]:
path = '/Users/alexandre/Desktop/Dataset'

In [42]:
actor.to_pickle(path+'/Actor.pkl')
final.to_pickle(path+'/Movie.pkl')

## Ethnie

In [7]:
characters_path = 'Character.pkl'
with open(characters_path, 'rb') as file:
    Characters = pickle.load(file)

In [5]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup

In [9]:
def get_ethnie(wikipedia_id):

    r = requests.get("https://www.wikidata.org/wiki/Special:Search"+wikipedia_id)
    soup = BeautifulSoup(r.content, "html.parser")
    etnies = soup.find('a', href=True, attrs={'data-serp-pos': '0', 'title': True})
    if etnies is not None:
        abc = etnies.find('span', class_='wb-itemlink-label').text
    else :
        abc = 0
    return abc

In [11]:
ethnie_id = pd.unique(Characters['Actor_ethnicity'].dropna())

In [13]:
ethnie = pd.Series("ethnie", index = ethnie_id)

In [15]:
a = 0
for i in tqdm(ethnie_id):
    ethnie.iloc[a] = get_ethnie(i)
    a +=1

100%|██████████| 479/479 [11:17<00:00,  1.41s/it]


In [21]:
ethnie[ethnie == 0].count()

48

In [25]:
ethnie.to_pickle('ethnie.pkl')